In [1]:
using FFTW, Random, Test

In [2]:
include("MacroUtils.jl")
using .MacroUtils: cleanup

Test data

In [3]:
Random.seed!(1234)
sample_vector_input = rand(32);
sample_tuple_input = Tuple(sample_vector_input);

In [4]:
function test_fft(fft)
    @test FFTW.fft(sample_vector_input) ≈ fft(sample_tuple_input)
end

test_fft (generic function with 1 method)

In [5]:
function fft_rec(a)
    n = length(a)
    @assert ispow2(n)
    isone(n) && return a
    n2 = n ÷ 2
    w = exp(-2 * pi * im / n)

    a1 = [a[k] for k in 1:2:n]
    a2 = [a[k] for k in 2:2:n]
    y1 = fft_rec(a1)
    y2 = fft_rec(a2)

    wk = [w^(k - 1) for k in 1:n2]
    z1 = [y1[k] + wk[k] * y2[k] for k in 1:n2]
    z2 = [y1[k] - wk[k] * y2[k] for k in 1:n2]
    return vcat(z1, z2)
end

fft_rec (generic function with 1 method)

In [6]:
fft_rec((10, 20, 30, 40))

4-element Vector{ComplexF64}:
 100.0 + 0.0im
 -20.0 + 20.0im
 -20.0 + 0.0im
 -20.0 - 20.0im

In [7]:
test_fft(fft_rec)

Test Passed

In [8]:
function fft_rec2(a, b)
    n = length(a)
    @assert ispow2(n)
    isone(n) && (b[1] = a[1]; return a)
    n2 = n ÷ 2
    w = exp(-2 * pi * im / n)

    a1 = [a[k] for k in 1:2:n]
    a2 = [a[k] for k in 2:2:n]
    y1 = Vector{Complex{Float64}}(undef, n2)
    y2 = Vector{Complex{Float64}}(undef, n2)
    fft_rec2(a1, y1)
    fft_rec2(a2, y2)
    for k in 1:n2
        wky = w^(k - 1) * y2[k]
        b[k]    = y1[k] + wky
        b[k+n2] = y1[k] - wky
    end
    return
end

fft_rec2 (generic function with 1 method)

In [9]:
function fft_rec2(a)
    b = Vector{Complex{Float64}}(undef, length(a))
    fft_rec2(a, b)
    return b
end

fft_rec2 (generic function with 2 methods)

In [10]:
fft_rec2((10.0, 20.0, 30.0, 40.0))

4-element Vector{ComplexF64}:
 100.0 + 0.0im
 -20.0 + 20.0im
 -20.0 + 0.0im
 -20.0 - 20.0im

In [11]:
test_fft(fft_rec2)

Test Passed

In [12]:
function fft_rec3(a, p, b)
    n = length(p)
    isone(n) && (b[1] = a[p[1]]; return a)
    n2 = n ÷ 2
    w = exp(-2 * pi * im / n)

    p1 = [p[k] for k in 1:2:n]
    p2 = [p[k] for k in 2:2:n]
    y1 = Vector{Complex{Float64}}(undef, n2)
    y2 = Vector{Complex{Float64}}(undef, n2)
    fft_rec3(a, p1, y1)
    fft_rec3(a, p2, y2)
    for k in 1:n2
        wky = w^(k - 1) * y2[k]
        b[k]    = y1[k] + wky
        b[k+n2] = y1[k] - wky
    end
    return
end

fft_rec3 (generic function with 1 method)

In [13]:
function fft_rec3(a)
    n = length(a)
    @assert ispow2(n)
    p = [k for k in 1:n]
    b = Vector{Complex{Float64}}(undef, length(a))
    fft_rec3(a, p, b)
    return b
end

fft_rec3 (generic function with 2 methods)

In [14]:
fft_rec3((10.0, 20.0, 30.0, 40.0))

4-element Vector{ComplexF64}:
 100.0 + 0.0im
 -20.0 + 20.0im
 -20.0 + 0.0im
 -20.0 - 20.0im

In [15]:
test_fft(fft_rec3)

Test Passed

In [16]:
function fft_rec4(a, p, b, m)
    n = length(p)
    isone(n) && (b[1] = a[p[1]]; return m)
    n2 = n ÷ 2
    w = exp(-2 * pi * im / n)

    p1 = [p[k] for k in 1:2:n]
    p2 = [p[k] for k in 2:2:n]
    y1 = Vector{Complex{Float64}}(undef, n2)
    m += 1
    y2 = Vector{Complex{Float64}}(undef, n2)
    m = fft_rec4(a, p1, y1, m)
    m = fft_rec4(a, p2, y2, m)
    for k in 1:n2
        wky = w^(k - 1) * y2[k]
        b[k]    = y1[k] + wky
        b[k+n2] = y1[k] - wky
    end
    return m
end

fft_rec4 (generic function with 1 method)

In [17]:
function fft_rec4(a)
    n = length(a)
    @assert ispow2(n)
    p = [k for k in 1:n]
    b = Vector{Complex{Float64}}(undef, length(a))
    m = fft_rec4(a, p, b, 1)
    return b
end

fft_rec4 (generic function with 2 methods)

In [18]:
fft_rec4((10.0, 20.0, 30.0, 40.0))

4-element Vector{ComplexF64}:
 100.0 + 0.0im
 -20.0 + 20.0im
 -20.0 + 0.0im
 -20.0 - 20.0im

In [19]:
test_fft(fft_rec4)

Test Passed

In [20]:
function fft_gen_impl1(p, b, m)
    n = length(p)
    es = Expr[]

    if isone(n)
        push!(es, :($b[1] = a[$(p[1])]))
        return es, m
    end

    n2 = n ÷ 2
    p1 = [p[k] for k in 1:2:n]
    p2 = [p[k] for k in 2:2:n]

    y1 = Symbol("y1_", m)
    y2 = Symbol("y2_", m)
    m += 1

    push!(es, :($y1 = Vector{Complex{Float64}}(undef, $n2)))
    push!(es, :($y2 = Vector{Complex{Float64}}(undef, $n2)))

    es1, m = fft_gen_impl1(p1, y1, m)
    es2, m = fft_gen_impl1(p2, y2, m)
    append!(es, es1, es2)

    w = exp(-2 * pi * im / n)

    for k in 1:n2
        wky = Symbol("wky", m)
        m += 1
        push!(es, :($wky = $(w^(k - 1)) * $y2[$k]))
        push!(es, :($b[$k] = $y1[$k] + $wky))
        push!(es, :($b[$(k + n2)] = $y1[$k] - $wky))
    end
    return es, m
end

fft_gen_impl1 (generic function with 1 method)

In [21]:
function fft_gen_impl1(n)
    @assert (n >= 1 && ispow2(n))
    p = [k for k in 1:n]
    b = Symbol("y0")
    es, m = fft_gen_impl1(p, b, 1)

    quote
        $b = Vector{Complex{Float64}}(undef, $n)
        $(es...)
        return $b
    end
end

fft_gen_impl1 (generic function with 2 methods)

In [22]:
fft_gen_impl1(8) |> cleanup

quote
    y0 = Vector{Complex{Float64}}(undef, 8)
    y1_1 = Vector{Complex{Float64}}(undef, 4)
    y2_1 = Vector{Complex{Float64}}(undef, 4)
    y1_2 = Vector{Complex{Float64}}(undef, 2)
    y2_2 = Vector{Complex{Float64}}(undef, 2)
    y1_3 = Vector{Complex{Float64}}(undef, 1)
    y2_3 = Vector{Complex{Float64}}(undef, 1)
    y1_3[1] = a[1]
    y2_3[1] = a[5]
    wky4 = (1.0 + 0.0im) * y2_3[1]
    y1_2[1] = y1_3[1] + wky4
    y1_2[2] = y1_3[1] - wky4
    y1_5 = Vector{Complex{Float64}}(undef, 1)
    y2_5 = Vector{Complex{Float64}}(undef, 1)
    y1_5[1] = a[3]
    y2_5[1] = a[7]
    wky6 = (1.0 + 0.0im) * y2_5[1]
    y2_2[1] = y1_5[1] + wky6
    y2_2[2] = y1_5[1] - wky6
    wky7 = (1.0 + 0.0im) * y2_2[1]
    y1_1[1] = y1_2[1] + wky7
    y1_1[3] = y1_2[1] - wky7
    wky8 = (6.123233995736766e-17 - 1.0im) * y2_2[2]
    y1_1[2] = y1_2[2] + wky8
    y1_1[4] = y1_2[2] - wky8
    y1_9 = Vector{Complex{Float64}}(undef, 2)
    y2_9 = Vector{Complex{Float64}}(undef, 2)
    y1_10 = Vector{Compl

In [23]:
@generated function fft_gen1(a::NTuple{N, Float64}) where {N}
    fft_gen_impl1(N)
end

fft_gen1 (generic function with 1 method)

In [24]:
fft_gen1((10.0, 20.0, 30.0, 40.0))

4-element Vector{ComplexF64}:
 100.0 + 0.0im
 -20.0 + 20.0im
 -20.0 + 0.0im
 -20.0 - 20.0im

In [25]:
test_fft(fft_gen1)

Test Passed

In [58]:
function fft_gen_impl2(p, b, m)
    n = length(p)
    es = Expr[]

    if isone(n)
        push!(es, :($(b[1]) = a[$(p[1])]))
        return es, m
    end

    n2 = n ÷ 2
    p1 = [p[k] for k in 1:2:n]
    p2 = [p[k] for k in 2:2:n]

    y1 = [Symbol("y1_", m, k) for k in 1:n2]
    y2 = [Symbol("y2_", m, k) for k in 1:n2]
    m += 1

    es1, m = fft_gen_impl2(p1, y1, m)
    es2, m = fft_gen_impl2(p2, y2, m)
    append!(es, es1, es2)

    w = exp(-2 * pi * im / n)

    for k in 1:n2
        wky = Symbol("wky", m)
        m += 1
        push!(es, :($wky = $(w^(k - 1)) * $(y2[k])))
        push!(es, :($(b[k]) = $(y1[k]) + $wky))
        push!(es, :($(b[k + n2]) = $(y1[k]) - $wky))
    end
    return es, m
end

fft_gen_impl2 (generic function with 2 methods)

In [59]:
function fft_gen_impl2(n)
    @assert (n >= 1 && ispow2(n))
    p = [k for k in 1:n]
    b = [Symbol("y0_", k) for  k in 1:n]
    es, m = fft_gen_impl2(p, b, 1)

    quote
        $(es...)
        return $(Expr(:vect, b...))
    end
end

fft_gen_impl2 (generic function with 2 methods)

In [60]:
fft_gen_impl2(8) |> cleanup

quote
    y1_31 = a[1]
    y2_31 = a[5]
    wky4 = (1.0 + 0.0im) * y2_31
    y1_21 = y1_31 + wky4
    y1_22 = y1_31 - wky4
    y1_51 = a[3]
    y2_51 = a[7]
    wky6 = (1.0 + 0.0im) * y2_51
    y2_21 = y1_51 + wky6
    y2_22 = y1_51 - wky6
    wky7 = (1.0 + 0.0im) * y2_21
    y1_11 = y1_21 + wky7
    y1_13 = y1_21 - wky7
    wky8 = (6.123233995736766e-17 - 1.0im) * y2_22
    y1_12 = y1_22 + wky8
    y1_14 = y1_22 - wky8
    y1_101 = a[2]
    y2_101 = a[6]
    wky11 = (1.0 + 0.0im) * y2_101
    y1_91 = y1_101 + wky11
    y1_92 = y1_101 - wky11
    y1_121 = a[4]
    y2_121 = a[8]
    wky13 = (1.0 + 0.0im) * y2_121
    y2_91 = y1_121 + wky13
    y2_92 = y1_121 - wky13
    wky14 = (1.0 + 0.0im) * y2_91
    y2_11 = y1_91 + wky14
    y2_13 = y1_91 - wky14
    wky15 = (6.123233995736766e-17 - 1.0im) * y2_92
    y2_12 = y1_92 + wky15
    y2_14 = y1_92 - wky15
    wky16 = (1.0 + 0.0im) * y2_11
    y0_1 = y1_11 + wky16
    y0_5 = y1_11 - wky16
    wky17 = (0.7071067811865476 - 0.7071067811865475

In [61]:
@generated function fft_gen2(a::NTuple{N, Float64}) where {N}
    fft_gen_impl2(N)
end

fft_gen2 (generic function with 1 method)

In [62]:
fft_gen2((10.0, 20.0, 30.0, 40.0))

4-element Vector{ComplexF64}:
 100.0 + 0.0im
 -20.0 + 20.0im
 -20.0 + 0.0im
 -20.0 - 20.0im

In [63]:
test_fft(fft_gen2)

Test Passed